In [1]:
# sed -i -e '/^[0-9]/d' Dragon*
# sed -i '/^$/d' Dragon*
# sed -i 1,3d Dragon*
#from __future__ import unicode_literals, print_function


import jparse
from __future__ import print_function, unicode_literals


with open('dragonball/Dragon_Ball_Super_001.srt', 'r') as f:
    data = f.read()

In [2]:
len(data)

19618

In [3]:
print(data[:1200])

＜史上最強の敵魔人ブウの出現により地球は　絶体絶命の危機に見舞われた＞

＜魔人ブウに立ち向かう孫　悟空と　ベジータ＞

＜サタンの呼び掛けで地球の　みんなと全宇宙の元気を集めたスーパー元気玉でついに　悟空は魔人ブウを倒したのだ＞

＜それから　半年＞

＜ドラゴンボールの力により人々から　ブウの記憶は消えしばらくの月日が流れた＞

（悟空）えっ？あっ。

（悟空）はぁ…。おぉ。お前 ベジータ みてぇに頑固なやつだな。 ヘッ。 よっ。

（悟天）お父さーん。
（悟空）うん？

（悟天）お弁当だよー。

（悟空）おう　悟天。
（悟空）何だ　学校は　もう終わったんか？

（悟天）今日は　日曜日だよ。

（悟空）休みか。　あっ。　ヒヒヒ。

（悟空）じゃあ　ちょっと　代わりにこいつ　運転してくれ。

（悟天）いいけど。

（悟空）おぉ！　弁当　弁当。おぉー！ いただきます。 いやー食ったー！ ごちそうさまでした 。よし。 ふっ


In [4]:
sentences = data.splitlines()

In [5]:
len(sentences)

622

In [6]:
sentences = [b for b in sentences if b]

In [7]:
print(sentences[0])

＜史上最強の敵魔人ブウの出現により地球は　絶体絶命の危機に見舞われた＞


In [8]:
type(sentences[0])

str

In [9]:
from collections import Counter

In [14]:
with open('data/characters.txt','r') as f:
    characters = f.read()

In [15]:
characters = characters.splitlines()

In [17]:
characters = [b for b in characters if b]

In [10]:
characters = ['悟天','悟空','サタン','魔人ブウ','ビルス','ウイス','ブウ','ビーデル','悟飯',
              'ピッコロ','トランクス','ブルマ','チチ','亀仙人','キビト神','老界王神','ベジータ']

In [18]:
character_dictionary = {}
for index, term in enumerate(characters):
    character_dictionary[index] = term


In [19]:
print(character_dictionary[0])

悟天


In [12]:
%load_ext autoreload

%autoreload 2


In [20]:
def create_corpus(sentences, terms):
    corpus = []
    for term in terms:
        corpus.append([b for b in sentences if term in b])
    return corpus

def annotate_training(corpus, dictionary, name):
    """
    Args: clean lower cased sentences, skills, entity category
    Returns: list of annotated sentences
    """
    train = []
    for b, _list in enumerate(corpus):
        for sentence in _list:
            train.extend([(int(sentence.index(dictionary[b])),
                           int(sentence.index(dictionary[b]) + len(dictionary[b])), 
                           name)])
    corpus = [item for sublist in corpus for item in sublist]
    return zip(corpus, train)
    

def removeNonAscii(data):
    """ use to remove stubborn non ascii characters from text; relevant in Python2
    example on dataframe: [removeNonAscii(b) for b in df['column']]
    example on json: [removeNonAscii(b['item']) for b in json]
    """
    return "".join(i for i in data if ord(i)<128)

In [21]:
sentences[0]

'\xef\xbc\x9c\xe5\x8f\xb2\xe4\xb8\x8a\xe6\x9c\x80\xe5\xbc\xb7\xe3\x81\xae\xe6\x95\xb5\xe9\xad\x94\xe4\xba\xba\xe3\x83\x96\xe3\x82\xa6\xe3\x81\xae\xe5\x87\xba\xe7\x8f\xbe\xe3\x81\xab\xe3\x82\x88\xe3\x82\x8a\xe5\x9c\xb0\xe7\x90\x83\xe3\x81\xaf\xe3\x80\x80\xe7\xb5\xb6\xe4\xbd\x93\xe7\xb5\xb6\xe5\x91\xbd\xe3\x81\xae\xe5\x8d\xb1\xe6\xa9\x9f\xe3\x81\xab\xe8\xa6\x8b\xe8\x88\x9e\xe3\x82\x8f\xe3\x82\x8c\xe3\x81\x9f\xef\xbc\x9e'

In [29]:
u'＜史上最強の敵魔人ブウの出現により地球は　絶体絶命の危機に見舞われた＞'

u'\uff1c\u53f2\u4e0a\u6700\u5f37\u306e\u6575\u9b54\u4eba\u30d6\u30a6\u306e\u51fa\u73fe\u306b\u3088\u308a\u5730\u7403\u306f\u3000\u7d76\u4f53\u7d76\u547d\u306e\u5371\u6a5f\u306b\u898b\u821e\u308f\u308c\u305f\uff1e'

In [23]:
corpus = create_corpus(sentences, characters)

In [24]:
train = annotate_training(corpus, character_dictionary, 'character')

In [25]:
print(train[0][0])

（悟天）お父さーん。


In [28]:
with open('data/db_train.txt', 'w') as f:
    for tup in train:
        f.write(str(tup)+'\n')

In [26]:
from __future__ import unicode_literals, print_function

import random
import spacy
import pandas as pd
import timeit
import ast


from spacy.lang.en import English
from spacy.gold import GoldParse, biluo_tags_from_offsets


from spacy.tagger import Tagger
from collections import OrderedDict

"""
Use: python train_ner_alpha.py training_corpus_path model_name
Example: python train_ner_alpha data/sample_corpus.txt sample_model
"""


def reformat_train_data(tokenizer, examples):
    """Reformat data to match JSON format"""
    output = []
    for i, (text, entity_offsets) in enumerate(examples):
        doc = tokenizer(text)
        ner_tags = biluo_tags_from_offsets(tokenizer(text), entity_offsets)
        words = [w.text for w in doc]
        tags = ['-'] * len(doc)
        heads = [0] * len(doc)
        deps = [''] * len(doc)
        sentence = (range(len(doc)), words, tags, heads, deps, ner_tags)
        output.append((text, [(sentence, [])]))
    return output


def train_ner(train_data, model_dir=None):
    
    nlp = English(pipeline=['tensorizer', 'ner'])
    get_data = lambda: reformat_train_data(nlp.tokenizer, train_data)
    optimizer = nlp.begin_training(get_data)
    for itn in range(50):
        print("iteration: "+str(itn))
        random.shuffle(train_data)
        losses = {}
        for raw_text, entity_offsets in train_data:
            doc = nlp.make_doc(raw_text)
            gold = GoldParse(doc, entities=entity_offsets)
            nlp.update(
                [doc], # Batch of Doc objects
                [gold], # Batch of GoldParse objects
                drop=0.5, # Dropout -- make it harder to memorise data
                sgd=optimizer, # Callable to update weights
                losses=losses)
        print(losses)
    print("Save to", model_dir)
    nlp.to_disk(model_dir)

def main(train, model_name):

    print("training name entity recognizer model...this may take a while")
    start = timeit.default_timer()
    train_ner(train, str(model_name)+"/")
    end = timeit.default_timer()-start
    print("Training complete with time {} minutes".format(end/60))
    
   

In [38]:
with open('data/db_train.txt') as f:
    train = list([b.strip('\n') for b in f])
        
train = [ast.literal_eval(b) for b in train]

In [46]:
print(train[0][0])

（悟天）お父さーん。


In [35]:
%load_ext autoreload

%autoreload 2

from ja import japanese

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
nlp = japanese.Japanese(pipeline=['tensorizer', 'ner'])


In [27]:
main(train, 'db')

training name entity recognizer model...this may take a while


TypeError: Argument 'string' has incorrect type (expected unicode, got str)